# Παίζοντας Atari με βαθιά ενισχυτική μάθηση



## Tensorboard και ngrok


In [9]:
!apt install apt-utils psmisc -y
!killall -9 ngrok
!rm -rf *ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mkdir -p ngrok-dir
!mv ngrok ngrok-dir/

# Στην επόμενη γραμμή βάλτε το δικό σας authentication token από το ngrok
!ngrok-dir/ngrok authtoken 1pFEw2CnsQ79NBUzgBj7reQI9Fl_4Ggvb9h6hi79gkvt97Dbq

LOG_DIR = 'tb_log/'""
!mkdir -p LOG_DIR

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('ngrok-dir/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print("Wait 10-15 seconds and then click the URL above to open TensorBoard")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
apt-utils is already the newest version (1.6.12ubuntu0.2).
psmisc is already the newest version (23.1-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.
--2021-03-15 20:40:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.226.107.193, 54.197.133.26, 34.234.81.220, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.226.107.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  16.9MB/s    in 0.8s    

2021-03-15 20:40:31 (16.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://

## Εγκατάσταση βιβλιοθήκης και gym

In [10]:
!pip install --upgrade pip
!pip install --upgrade stable_baselines3[extra]
# we need a specific version of gym because of this issue: https://github.com/DLR-RM/stable-baselines3/issues/294
!pip install gym==0.17.3

## Ορισμός παιχνιδιού

Το παιχνίδι που αντιστοιχεί στην ομάδα μας είναι το Boxing. 
- v0 vs v4: v0 has repeat_action_probability of 0.25 (meaning 25% of the time the previous action will be used instead of the new action), while v4 has 0 (always follow your issued action)
- Deterministic: a fixed frameskip of 4, while for the env without Deterministic, frameskip is sampled from [2,4]
- There is also NoFrameskip-v4 with no frame skip and no action repeat stochasticity.



In [11]:
atari_env_name='Boxing-v0'


## Δημιουργία περιβάλλοντος

In [12]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

In [13]:
# Με τις συναρτήσεις που ακολουθούν κάνουμε την ίδια προεπεξεργασία με την DeepMind

# Here we are also multi-worker training (n_envs=4 => 4 environments), The model must support Multi Processing. To DQN δεν επιτρέπει multi envs
env = make_atari_env(atari_env_name, n_envs=1)
# Frame-stacking with 4 frames
env = VecFrameStack(env, n_stack=4)
# Test environment must be unique
test_env = make_atari_env(atari_env_name, n_envs=1)
# Frame-stacking with 4 frames
test_env = VecFrameStack(test_env, n_stack=4)





## Εκπαίδευση

Θα εκπαιδεύσουμε ένα δίκτυο deep q-learning (QRDQN) όπως αυτό της Deepmind. 

In [14]:
import datetime # For filenames while logging
!pip install sb3-contrib

In [15]:
!pip install sb3-contrib
from sb3_contrib import QRDQN






In [ ]:
model_name='QRDQN-CNNPolicy-v0'
time_stamp=datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")
model_log= LOG_DIR + model_name + time_stamp
qrdqn_model = QRDQN('CnnPolicy', env, verbose=1, tensorboard_log=model_log, buffer_size=100000)

In [ ]:
#max_steps=1200000
#qrdqn_model.learn(total_timesteps=max_steps)

## Εκτίμηση απόδοσης

In [ ]:
#from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
#mean_reward, std_reward = evaluate_policy(qrdqn_model, test_env, n_eval_episodes=20)
#print(f"Eval reward: {mean_reward} (+/-{std_reward})")

## Σώσιμο εκπαιδευμένου μοντέλου

Το μοντέλο θα αποθηκευθεί ως zip και μπορείτε να το κατεβάσετε τοπικά από το δεξί sidebar του Kaggle στο "Data" και μετά στο ellipsis menu πάνω στο filename.

## Φόρτωση εκπαιδευμένου μοντέλου



In [26]:
!wget --no-check-certificate 'https://www.dropbox.com/s/w8xj6s6cde5nch0/qrdqn_cnn_sketo_v0_best.zip?dl=1' -O qrdqn_best

--2021-03-15 20:48:20--  https://www.dropbox.com/s/w8xj6s6cde5nch0/qrdqn_cnn_sketo_v0_best.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/w8xj6s6cde5nch0/qrdqn_cnn_sketo_v0_best.zip [following]
--2021-03-15 20:48:20--  https://www.dropbox.com/s/dl/w8xj6s6cde5nch0/qrdqn_cnn_sketo_v0_best.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc159c25eb5f6ac558e6a484db50.dl.dropboxusercontent.com/cd/0/get/BKzUePCQAmXKOVinDxg8X7FY2rMXexnv8PFnaLpagkP34ac_HZHaUXoVPyRlIltIPXDVnRzyRiCaOFEX8CUbK0pRKxvGDyacKbgVb_tX2Rz5X1hGZu7EiV5Qb9PJuQP6i-AmdpNutXNTDpm9MFIAv6fc/file?dl=1# [following]
--2021-03-15 20:48:21--  https://uc159c25eb5f6ac558e6a484db50.dl.dropboxusercontent.com/cd/0/get/BKzUePCQAmXKOVinDxg8X7FY2rMXexnv8PFnaLpagkP

In [30]:
!pip install sb3-contrib
from sb3_contrib import QRDQN

qrdqn_model = QRDQN.load("qrdqn_best", verbose=1)

## Συνέχιση της εκπαίδευσης

In [32]:
# Προσοχή, το n_envs πρέπει να είναι πάντα το ίδιο (αυτό με το οποίο έγινε η αρχική εκπαίδευση του μοντέλου)
new_env = make_atari_env(atari_env_name, n_envs=1, seed=0)
# Frame-stacking with 4 frames
new_env = VecFrameStack(new_env, n_stack=4)
# Το μοντέλο δεν έχει μαζί του το περιβάλλον, πρέπει να του το αναθέσουμε ξανά.
qrdqn_model.set_env(new_env)

Wrapping the env in a VecTransposeImage.


## Καταγραφή video του actual gameplay του πράκτορα

Ωραία τα διαγράμματα, ωραίαο το TensorBoard, αλλά θέλουμε οπωσδήποτε να δούμε τον πράκτορα να παίζει το παιχνίδι!

In [33]:
!apt-get install ffmpeg freeglut3-dev xvfb  -y # For visualization

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.


In [34]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [35]:
video_folder = '/kaggle/working/videos'

In [36]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [37]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(eval_env, model, video_length=500, prefix='', video_folder=video_folder):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action =  [eval_env.action_space.sample()] 
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [38]:
record_video(test_env, qrdqn_model, video_length=10000, prefix='qrdqn_boxing')

Saving video to  /kaggle/working/videos/qrdqn_boxing-step-0-to-step-10000.mp4


In [39]:
show_videos(video_path = video_folder, prefix='qrdqn_boxing')

Μπορείτε να κατεβάσετε το video με hover πάνω του, κάτω δεξιά στο ellipsis menu και "Download".